<a href="https://colab.research.google.com/github/AndresChapo/Collect-historic-prices-Bitmex-API/blob/main/Collect_5m_historic_price_from_Bitmex_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install bitmex

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 133kB 26.7MB/s 
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none-any.whl size=18121 sha256=cc162c2f3bca7dafb9ebc40b68cd5d9332fb3963d52167ab71fe2cfbb8a1a651
  Stored in directory: /root/.cache/pip/wheels/bb/af/c9/b6e9fb5f9b2470e4ed2a7241c9ab3a8cdd3bc8555ae02ca2e6
Successfully built strict-rfc3339


In [ ]:
import bitmex
import requests, json
import datetime
#import numpy as np
import csv, io
client = bitmex.bitmex()
#client = bitmex.bitmex(api_key=<YOUR API KEY>, api_secret=<YOUR API SECRET>)

/usr/local/lib/python3.7/dist-packages/swagger_spec_validator/validator20.py:53: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  ref_dict['$ref'], '/'.join(path),


In [ ]:
# Ask for permitions to save data into google drive
from google.colab import drive
 
drive.mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Just for experiments

#result = client.Quote.Quote_get(symbol="XBTUSD", reverse=True, count=1).result()
#print("bidPrice: " + str(result[0][0]['bidPrice']) + " - bidSize: " + str(result[0][0]['bidSize']))
#print("askPrice: " + str(result[0][0]['askPrice']) + " - askSize: " + str(result[0][0]['askSize']))

#client.Instrument.Instrument_get(symbol='XBTUSD').result()
#client.Quote.Quote_get(symbol="XBTUSD", startTime=datetime.datetime(2020, 1, 1)).result()
#client.Quote.Quote_get(symbol="XBTUSD", endTime=datetime.datetime.utcnow()).result()
#client.Position.Position_get(filter=json.dumps({'symbol': 'XBTUSD'})).result() #Requiere estar autenticado con un API KEY
#client.Instrument.Instrument_get(filter=json.dumps({'rootSymbol': 'XBT'})).result()

#first_page = client.Instrument.Instrument_get(count=2).result()
#second_page = client.Instrument.Instrument_get(count=2, start=2).result()
#print(str(first_page) + str(second_page))


#result = client.Settlement.Settlement_get(symbol="XBT:perpetual", count=5, reverse=True).result()
#for i in result[0]:
  #print("timestamp: " + str(i['timestamp']) + " - settledPrice: " + str(i['settledPrice']))
  #print(str(i['timestamp']))

#from itertools import chain
#from collections import defaultdict

# lista = []
# lista.append( {'bookA': 1, 'bookB': 2, 'bookC': 3})
# lista.append( {'bookA': 4, 'bookB': 5, 'bookC': 6})

#lista.extend([dict1])
#lista.extend([dict2])
#for k, v in dict3.items():

# print(lista)

[{'bookA': 1, 'bookB': 2, 'bookC': 3}, {'bookA': 4, 'bookB': 5, 'bookC': 6}]


Aca empieza la programación real

In [ ]:

hoy = datetime.datetime.utcnow()

dia = []
dia.append(client.Trade.Trade_getBucketed(symbol='XBT:perpetual', count=290, binSize="5m",startTime=hoy.replace(day=(hoy.day - 1)),endTime=hoy).result())
for x in range(1, 7):
  dia.append(client.Trade.Trade_getBucketed(symbol='XBT:perpetual', count=290, binSize="5m",startTime=hoy.replace(day=(hoy.day - (x+1) )),endTime=hoy.replace(day=(hoy.day - x))).result())



In [ ]:

contar = 0
for diax in reversed(dia):
  for i in diax[0]:
    print("timestamp: " + str(i['timestamp']) + " - closeprice: " + str(i['close'])+ " - volume: " + str(i['volume']))
    contar +=1
print(contar)



Streaming output truncated to the last 5000 lines.
timestamp: 2021-01-11 15:30:00+00:00 - closeprice: 32624.5 - volume: 251957
timestamp: 2021-01-11 15:35:00+00:00 - closeprice: 32184.5 - volume: 1611432
timestamp: 2021-01-11 15:40:00+00:00 - closeprice: 32444.0 - volume: 631503
timestamp: 2021-01-11 15:45:00+00:00 - closeprice: 31790.0 - volume: 1864400
timestamp: 2021-01-11 15:50:00+00:00 - closeprice: 31416.5 - volume: 2342989
timestamp: 2021-01-11 15:55:00+00:00 - closeprice: 31967.5 - volume: 1235577
timestamp: 2021-01-11 16:00:00+00:00 - closeprice: 31575.5 - volume: 636917
timestamp: 2021-01-11 16:05:00+00:00 - closeprice: 31448.0 - volume: 2805957
timestamp: 2021-01-11 16:10:00+00:00 - closeprice: 31140.5 - volume: 2598595
timestamp: 2021-01-11 16:15:00+00:00 - closeprice: 30748.5 - volume: 574337
timestamp: 2021-01-11 16:20:00+00:00 - closeprice: 30795.5 - volume: 1669954
timestamp: 2021-01-11 16:25:00+00:00 - closeprice: 31474.5 - volume: 3677223
timestamp: 2021-01-11 16:30:0

In [ ]:
# Save as CSV
with io.open('/content/drive/MyDrive/Colab Notebooks/precio_historico_5m_lastMonth.csv', 'w') as csvfile:
    campos = ['timestamp', 'closeprice', 'volume']
    writer = csv.DictWriter(csvfile, fieldnames=campos)

    writer.writeheader()
    for diax in reversed(dia):
      for i in diax[0]:
        writer.writerow({'timestamp': i['timestamp'], 'closeprice': i['close'], 'volume': i['volume']})


In [ ]:
# Open to test the CSV

with io.open('/content/drive/MyDrive/Colab Notebooks/precio_historico_5m_lastMonth.csv','r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    a=0
    for row in csv_reader:
        print(row)
        a+=1
print(a)

Streaming output truncated to the last 5000 lines.
['2021-01-11 15:30:00+00:00', '32624.5', '251957']
['2021-01-11 15:35:00+00:00', '32184.5', '1611432']
['2021-01-11 15:40:00+00:00', '32444.0', '631503']
['2021-01-11 15:45:00+00:00', '31790.0', '1864400']
['2021-01-11 15:50:00+00:00', '31416.5', '2342989']
['2021-01-11 15:55:00+00:00', '31967.5', '1235577']
['2021-01-11 16:00:00+00:00', '31575.5', '636917']
['2021-01-11 16:05:00+00:00', '31448.0', '2805957']
['2021-01-11 16:10:00+00:00', '31140.5', '2598595']
['2021-01-11 16:15:00+00:00', '30748.5', '574337']
['2021-01-11 16:20:00+00:00', '30795.5', '1669954']
['2021-01-11 16:25:00+00:00', '31474.5', '3677223']
['2021-01-11 16:30:00+00:00', '31236.5', '138269']
['2021-01-11 16:35:00+00:00', '30627.5', '2069121']
['2021-01-11 16:40:00+00:00', '30479.5', '2724462']
['2021-01-11 16:45:00+00:00', '30421.0', '1110394']
['2021-01-11 16:50:00+00:00', '30602.5', '1003395']
['2021-01-11 16:55:00+00:00', '31238.5', '1231894']
['2021-01-11 17:00